## Driver

In [3]:
from qiskit_nature.drivers import PySCFDriver
from qiskit_nature.drivers.qmolecule import QMolecule

# molecule = "H .0 .0 .0; H .0 .0 0.739"
molecule = 'Li 0.0 0.0 0.0; H 0.0 0.0 1.5474'
driver = PySCFDriver(atom = molecule)
qmolecule = driver.run()

## Electronic Structure and Freeze Core Qubits and Remove Orbitals

In [4]:
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers import FreezeCoreTransformer

problem = ElectronicStructureProblem(driver, q_molecule_transformers = [FreezeCoreTransformer(freeze_core = True, remove_orbitals = [3, 4])])
# same for remove_orbitals = [4]

second_quantized_operator = problem.second_q_ops()

# Main Operator = Hamiltonian
main_operator = second_quantized_operator[0]

# print(main_operator)

In [95]:
# data = problem.molecule_data_transformed
# print(data)

## Qubit Converter (Mapping) and Z2Symmetry Reduction

In [5]:
from qiskit_nature.mappers.second_quantization import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.converters.second_quantization.qubit_converter import QubitConverter

# mapping = "JordanWignerMapper"
mapping = "ParityMapper"
# mapping = "BravyiKitaevMapper"

if mapping == "ParityMapper":
    mapping = ParityMapper()
elif mapping == "JordanWignerMapper":
    mapping = JordanWignerMapper()
elif mapping == "BravyiKitaevMapper":
    mapping = BravyiKitaevMapper()
    



# converter = QubitConverter(mapper = mapping, two_qubit_reduction = False)
# converter = QubitConverter(mapper = mapping, two_qubit_reduction = True, z2symmetry_reduction = [1, 1, 1, 1])
converter = QubitConverter(mapper = mapping, two_qubit_reduction = True, z2symmetry_reduction = [1])

# Possible Z2Symmetries 

# [1, 1, 1, 1], [1, 1, 1, -1], [1, 1, -1, 1], 
# [1, 1, -1, -1], [1, -1, 1, 1], [1, -1, 1, -1], 
# [1, -1, -1, 1], [1, -1, -1, -1], [-1, 1, 1, 1], [-1, 1, 1, -1], 
# [-1, 1, -1, 1], [-1, 1, -1, -1], [-1, -1, 1, 1], [-1, -1, 1, -1], 
# [-1, -1, -1, 1], [-1, -1, -1, -1]

In [97]:
# print(converter.z2symmetry_reduction)
# print(converter.two_qubit_reduction)
# print(converter.z2symmetries)

In [6]:
# Mapping fermionic operators to qubit operators
num_particles = (problem.molecule_data_transformed.num_alpha, problem.molecule_data_transformed.num_beta)

print(num_particles)

(1, 1)


In [7]:
qubit_operator = converter.convert(main_operator, num_particles = num_particles)
# print(qubit_operator)

## Z2Symmetries and Remove Orbitals

In [65]:
from qiskit.opflow.primitive_ops import Z2Symmetries

sym = Z2Symmetries.find_Z2_symmetries(qubit_operator)
print(sym)

Z2 symmetries:
Symmetries:
IIIIZIII
IIZZIIZZ
IZIIIZII
ZIIIIIII
Single-Qubit Pauli X:
IIIIXIII
IIIIIIIX
IIIIIXII
XIIIIIII
Cliffords:
0.7071067811865475 * IIIIZIII
+ 0.7071067811865475 * IIIIXIII
0.7071067811865475 * IIZZIIZZ
+ 0.7071067811865475 * IIIIIIIX
0.7071067811865475 * IZIIIZII
+ 0.7071067811865475 * IIIIIXII
0.7071067811865475 * ZIIIIIII
+ 0.7071067811865475 * XIIIIIII
Qubit index:
[3, 0, 2, 7]
Tapering values:
  - Possible values: [1, 1, 1, 1], [1, 1, 1, -1], [1, 1, -1, 1], [1, 1, -1, -1], [1, -1, 1, 1], [1, -1, 1, -1], [1, -1, -1, 1], [1, -1, -1, -1], [-1, 1, 1, 1], [-1, 1, 1, -1], [-1, 1, -1, 1], [-1, 1, -1, -1], [-1, -1, 1, 1], [-1, -1, 1, -1], [-1, -1, -1, 1], [-1, -1, -1, -1]


In [100]:
# remove_ob = FreezeCoreTransformer.transform(True, molecule_data = qubit_operator)
# print(remove_ob)

In [66]:
from qiskit.opflow.primitive_ops import Z2Symmetries

# sym = Z2Symmetries.find_Z2_symmetries(qubit_operator)
sym = Z2Symmetries.symmetries
print(sym)

## Initial State (Hartree Fock)

In [8]:
from qiskit_nature.circuit.library import HartreeFock

num_particles = (problem.molecule_data_transformed.num_alpha, problem.molecule_data_transformed.num_beta)

num_spin_orbitals = 2 * problem.molecule_data_transformed.num_molecular_orbitals
print(num_spin_orbitals)

initial_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(initial_state)

6
     ┌───┐
q_0: ┤ X ├
     ├───┤
q_1: ┤ X ├
     └───┘
q_2: ─────
          
q_3: ─────
          


## Ansatz

In [80]:
from qiskit.circuit.library import TwoLocal
from qiskit_nature.circuit.library import UCCSD, PUCCD, SUCCD

# Available Ansatz

# ansatz = "TwoLocal"
ansatz = "UCCSD"
  
if ansatz == "UCCSD":
    ansatz = UCCSD(qubit_converter = converter, 
                   num_particles = num_particles, 
                   num_spin_orbitals = num_spin_orbitals, 
                   initial_state = initial_state)
elif ansatz == "PUCCD":
    ansatz = PUCCD(qubit_converter = converter, 
                   num_particles = num_particles, 
                   num_spin_orbitals = num_spin_orbitals, 
                   initial_state = initial_state)
elif ansatz == "SUCCD":
    ansatz = SUCCD(qubit_converter = converter, 
                   num_particles = num_particles, 
                   num_spin_orbitals = num_spin_orbitals, 
                   initial_state = initial_state)

elif ansatz == "TwoLocal":
    # Ansatz Parameters
    # Single qubit rotations
    rotation_blocks = ['ry', 'rz']
    
    # Entanglement gates
    entanglement_blocks = 'cx'
    
    # Type of entanglement
    entanglement = "full"    # or "linear"
    
    # Repitition of ansatz
    repititions = 6    # arbitrary number
    
    # Skip final rotation layer
    skip_final_rotation_layer = True
    
    # Ansatz
    ansatz = TwoLocal(num_qubits = qubit_operator.num_qubits, 
                      rotation_blocks = rotation_blocks, 
                      entanglement_blocks = entanglement_blocks, 
                      entanglement = entanglement, 
                      reps = repititions, 
                      skip_final_rotation_layer = skip_final_rotation_layer)
    
    ansatz.compose(initial_state, front = True, inplace = True)

# elif ansatz == "Custom":
#     # Add custom ansatz

## Backend

In [81]:
from qiskit import Aer
backend = Aer.get_backend('statevector_simulator')

## Optimizer

In [82]:
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP

optimizer = "COBYLA"

if optimizer == "COBYLA":
    optimizer = COBYLA(maxiter = 500)
elif optimizer == "L_BFGS_B":
    optimizer = L_BFGS_B(maxiter = 500)
elif optimizer == "SPSA":
    optimizer = SPSA(maxiter = 500)
elif optimizer == "SLSQP":
    optimizer = SLSQP(maxiter = 500)

## VQE and Inital Parameters of Ansatz

In [83]:
from qiskit.algorithms import VQE
from IPython.display import display, clear_output

# Initial Parameters of Ansatz
try:
    initial_point = [0.01] * len(ansatz.ordered_parameters)
except:
    initial_point = [0.01] * ansatz.num_parameters
    
algorithm = VQE(ansatz = ansatz, 
                optimizer = optimizer, 
                initial_point = initial_point, 
                quantum_instance = backend)
#                 callback = callback)

result = algorithm.compute_minimum_eigenvalue(qubit_operator)
print(result)

{   'aux_operator_eigenvalues': None,
    'cost_function_evals': 50,
    'eigenstate': array([ 5.91762316e-17+1.48939653e-17j,  2.78485212e-17+5.70741527e-17j,
        2.24812922e-16-1.26677548e-16j,  9.93195727e-01-1.40409528e-16j,
       -2.66387178e-19+2.71603126e-17j,  2.43882596e-17+7.02606815e-18j,
       -1.16702301e-02+6.46796850e-17j, -2.62809101e-16-1.78890544e-16j,
        2.82921619e-17+1.22663473e-17j,  7.17699910e-17+6.68846041e-17j,
       -1.17624440e-16+1.77933793e-16j, -2.23390020e-16-2.17131450e-17j,
       -2.98024049e-18-3.92864974e-18j, -3.11228916e-17-1.84250537e-17j,
       -3.06668730e-02+9.57778558e-17j, -7.17699910e-17-4.97875644e-18j,
        3.24877661e-17+1.90917598e-17j,  6.43696931e-17-5.88784672e-17j,
       -2.27002209e-17+2.09580191e-17j,  8.46298471e-17-8.95410157e-17j,
       -1.73038855e-17+2.62274499e-17j, -2.22537737e-17-2.77555756e-17j,
        6.37311415e-18-2.35691981e-17j,  6.57962091e-18+1.14967359e-17j,
        1.82273520e-17-6.87863180e-17

In [ ]:
# from qiskit.algorithms import VQE
# from qiskit.chemistry.algorithms import VQEAdapt
# from IPython.display import display, clear_output

# # Initial Parameters of Ansatz
# try:
#     initial_point = [0.01] * len(ansatz.ordered_parameters)
# except:
#     initial_point = [0.01] * ansatz.num_parameters
    
# # algorithm = VQE(ansatz = ansatz, 
# #                 optimizer = optimizer, 
# #                 initial_point = initial_point, 
# #                 quantum_instance = backend)
# # #                 callback = callback)

# algorithm = VQEAdapt(qubit_operator, 
#                      var_form_base = ansatz, 
#                      optimizer = optimizer, 
#                      initial_point = initial_point, 
#                      quantum_instance = backend)
# #                 callback = callback)

# result = algorithm.compute_minimum_eigenvalue(qubit_operator)
# print(result)

## Exact Energy

In [29]:
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 

def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
# print(result_exact)

Exact electronic energy -1.0856642363219928
